In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_03 import *

## DataBunch/Learner

In [3]:
x_train,y_train,x_valid,y_valid = get_data()
train_ds,valid_ds = Dataset(x_train, y_train), Dataset(x_valid, y_valid) 
nh,bs = 50,64
c = y_train.max() + 1
loss_func = F.cross_entropy

Factor out the connected pieces of info out of the fit() argument list

`fit(epochs, model, loss_func, opt, train_dl, valid_dl)`

Let's replace it with something that looks like this:

`fit(1, learn)`

This will allow us to tweak what's happening inside the training loop in other places of the code because the `Learner` object will be mutable, so changing any of its attribute elsewhere will be seen in our training loop.

In [4]:
get_dls??

In [10]:
#export
class DataBunch():
    def __init__(self, train_dl, valid_dl, c):
        self.train_dl, self.valid_dl, self.c = train_dl, valid_dl, c
        
    @property
    def train_ds(self): return self.train_dl.dataset
        
    @property
    def valid_ds(self): return self.valid_dl.dataset

In [11]:
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [35]:
#export
def get_model(data, lr=0.5):
    m = data.train_ds.x.shape[1]
    model = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh, int(data.c)))
    return model, optim.SGD(model.parameters(), lr=lr)

class Learner():
    def __init__(self, model, opt, loss_func, data):
        self.model, self.opt, self.loss_func, self.data = model, opt, loss_func, data

In [36]:
learn = Learner(*get_model(data), loss_func, data)

In [53]:
def fit(epochs, learn):
    for epoch in range(epochs):
        learn.model.train()
        for xb, yb in learn.data.train_dl:
            loss = learn.loss_func(learn.model(xb), yb)
            loss.backward()
            learn.opt.step()
            learn.opt.zero_grad()

        learn.model.eval()
        with torch.no_grad():
            tot_loss, tot_acc = 0., 0.
            for xb, yb in learn.data.valid_dl:
                pred = learn.model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc += accuracy(pred, yb)
        nv = len(learn.data.valid_dl)
        print('2o3', epoch, tot_loss/nv, tot_acc/nv)
    return tot_loss/nv, tot_acc/nv

In [54]:
loss,acc = fit(1, learn)

2o3 0 tensor(0.1172) tensor(0.9660)


## CallbackHandler

This was our training loop (without validation) from the previous notebook, with the inner loop contents factored out:

```python
def one_batch(xb,yb):
    pred = model(xb)
    loss = loss_func(pred, yb)
    loss.backward()
    opt.step()
    opt.zero_grad()
    
def fit():
    for epoch in range(epochs):
        for b in train_dl: one_batch(*b)
```

Add callbacks so we can remove complexity from loop, and make it flexible:

In [122]:
def one_batch(xb, yb, cb):
    if not cb.begin_batch(xb,yb): return
    loss = cb.learn.loss_func(cb.learn.model(xb), yb)
    if not cb.after_loss(loss): 
        print(':O')
        return
    loss.backward()
    if cb.after_backward(): cb.learn.opt.step()
    if cb.after_step(): cb.learn.opt.zero_grad()

def all_batches(dl, cb):
    for xb,yb in dl:
        one_batch(xb, yb, cb)
        if cb.do_stop(): return

def fit(epochs, learn, cb):
    if not cb.begin_fit(learn): return
    for epoch in range(epochs):
        if not cb.begin_epoch(epoch): continue
        all_batches(learn.data.train_dl, cb)
        
        if cb.begin_validate():
            with torch.no_grad(): all_batches(learn.data.valid_dl, cb)
        if cb.do_stop() or not cb.after_epoch(): break
    cb.after_fit()

In [127]:
class Callback():
    def begin_fit(self, learn):
        self.learn = learn
        return True
    
    def after_fit(self): return True
    
    def begin_epoch(self, epoch):
        self.epoch = epoch
        return True
    
    def begin_validate(self): return True
    
    def after_epoch(self): return True
    
    def begin_batch(self, xb, yb):
        self.xb, self.yb = xb, yb
        return True
    
    def after_loss(self, loss):
        self.loss = loss
        print('self.learn.in_train', self.learn.in_train)
        print(True if self.learn.in_train else False)
        return (True if self.learn.in_train else False)
    
    def after_backward(self): return True
    
    def after_step(self): return True

In [131]:
class CallbackHandler():
    def __init__(self, cbs=None):
        self.cbs = cbs

    def begin_fit(self, learn):
        self.learn = learn
        self.learn.stop = False
        res = True
        for cb in self.cbs: res = res and cb.begin_fit(learn)
        return res

    def after_fit(self):
        res = True
        for cb in self.cbs: res = res and cb.after_fit()
        return res
    
    def begin_epoch(self, epoch):
        self.epoch, self.learn.in_train = epoch, True
        self.learn.model.train()
        res = True
        for cb in self.cbs: res and cb.begin_epoch(epoch)
        return True

    def begin_validate(self):
        self.learn.in_train = False
        self.learn.model.eval()
        res = True
        for cb in self.cbs: res and cb.begin_validate()
        return res

    def after_epoch(self):
        res = True
        for cb in self.cbs: res and cb.after_epoch()
        return res
    
    def begin_batch(self, xb, yb):
        res = True
        for cb in self.cbs: res and cb.begin_batch(xb, yb)
        return res

    def after_loss(self, loss):
        res = True
        print([cb.after_loss(loss) for cb in self.cbs])
        for cb in self.cbs: res and cb.after_loss(loss)
        return res

    def after_backward(self):
        res = True
        for cb in self.cbs: res and cb.after_backward()
        return res

    def after_step(self):
        res = True
        for cb in self.cbs: res and cb.after_step()
        return res
    
    def do_stop(self):
        try: 
            return self.learn.stop
        finally: return self.learn.stop

In [132]:
class TestCallback(Callback):
    def begin_fit(self, learn):
        self.learn = learn
        self.n_iters = 1
        return True
        
    def after_step(self):
        self.n_iters += 1
        print(self.n_iters)
        if self.n_iters >= 10: self.learn.stop = True
        return True

In [133]:
fit(1, learn, cb=CallbackHandler([TestCallback()]))

self.learn.in_train True
True
[True]
self.learn.in_train True
True
2
self.learn.in_train True
True
[True]
self.learn.in_train True
True
3
self.learn.in_train True
True
[True]
self.learn.in_train True
True
4
self.learn.in_train True
True
[True]
self.learn.in_train True
True
5
self.learn.in_train True
True
[True]
self.learn.in_train True
True
6
self.learn.in_train True
True
[True]
self.learn.in_train True
True
7
self.learn.in_train True
True
[True]
self.learn.in_train True
True
8
self.learn.in_train True
True
[True]
self.learn.in_train True
True
9
self.learn.in_train True
True
[True]
self.learn.in_train True
True
10
self.learn.in_train False
False
[False]
self.learn.in_train False
False


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

This is roughly how fastai does it now (except that the handler can also change and return `xb`, `yb`, and `loss`). But let's see if we can make things simpler and more flexible, so that a single class has access to everything and can change anything at any time. The fact that we're passing `cb` to so many functions is a strong hint they should all be in the same class!

## Runner

In [ ]:
#export
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()

class Callback():
    #
    def set_runner#
    def __getattr__#
    #
    def name#
        #
        return camel2snake#

This first callback is reponsible to switch the model back and forth in training or validation mode, as well as maintaining a count of the iterations, or the percentage of iterations ellapsed in the epoch.

In [ ]:
#export
class TrainEvalCallback#
    def begin_fit#
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch#
        #
        #
        #
        
    def begin_epoch#
        #
        #self.model.train()
        #

    def begin_validate#
        #
        #

We'll also re-create our TestCallback - but note this doesn't actually work right yet (can you see why?) We'll fix it in notebook 05b.

In [ ]:
# Not working!
class TestCallback(Callback):
    _order=1
    def after_step(self):
        if self.n_iter>=10: return True

In [ ]:
cbname = 'TrainEvalCallback'
camel2snake(cbname)

'train_eval_callback'

In [ ]:
TrainEvalCallback()#

'train_eval'

In [ ]:
#export
from ## import *

def listify(o):
    if o is None: #
    if isinstance(o, list): #
    if isinstance(o, str): #
    if isinstance(o, Iterable): #
    return #

In [ ]:
#export
class Runner():
    def __init__#
        #
        for#
            #
            #
            #
        self.stop,self.cbs = #

    #
    def opt#
    #
    def model#
    #
    def loss_func#
    #
    def data#

    def one_batch#
        self.xb,self.yb = xb,yb
        #
        self.pred = self.model(self.xb)
        #
        self.loss = self.loss_func(self.pred, self.yb)
        #
        self.loss.backward()
        #
        self.opt.step()
        #
        self.opt.zero_grad()

    def all_batches#
        self.iters =#
        for #
            #
            self.one_batch(xb, yb)
            #
        self.stop=False

    def fit#
        #

        try:
            #
            #
            for epoch in range(epochs):
                #
                #

                #
                    #
                #
            
        finally:
            #
            #

    def __call__#
        #
            #f = getattr(cb, cb_name, None)
            #
        #

Third callback: how to compute metrics.

In [ ]:
#export
class AvgStats():
    def __init__#
    
    def reset#
        #
        #
        
    #
    def all_stats#
    #
    def avg_stats#
    
    def __repr__#
        if not self.count: return ""
        return #

    def accumulate#
        #
        #
        #
        #for
            #

class AvgStatsCallback#
    def __init__#
        self.train_stats,self.valid_stats = AvgStats(metrics,True),AvgStats(metrics,False)
        
    def begin_epoch#
        #
        #
        
    def after_loss#
        #
        #
    
    def after_epoch#
        #
        print(#

In [ ]:
learn = Learner(*get_model(data), loss_func, data)

In [ ]:
stats = AvgStatsCallback([accuracy])
run = Runner(cbs=stats)

In [ ]:
run.fit(2, learn)

train: [0.31685572265625, tensor(0.9033)]
valid: [0.15482904052734375, tensor(0.9553)]
train: [0.143599248046875, tensor(0.9567)]
valid: [0.116745849609375, tensor(0.9652)]


In [ ]:
loss,acc = stats.valid_stats.avg_stats
assert acc>0.9
loss,acc

(0.116745849609375, tensor(0.9652))

In [ ]:
#export
from ## import partial

In [ ]:
acc_cbf = #

In [ ]:
run = Runner(cb_funcs=acc_cbf)

In [ ]:
run.fit(1, learn)

train: [0.108607373046875, tensor(0.9666)]
valid: [0.131622998046875, tensor(0.9607)]


Using Jupyter means we can get tab-completion even for dynamic code like this! :)

In [ ]:
run.avg_stats.valid_stats.avg_stats

[0.131622998046875, tensor(0.9607)]

## Export

In [ ]:
!python notebook2script.py 04_callbacks.ipynb